<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/enhancing_rag_with_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

In [ ]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough

try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

# 通过构建和利用知识图谱来提高基于 RAG 的应用程序准确性
## 使用 Neo4j 和 LangChain 在 RAG 应用程序中构建和检索知识图谱信息的实用指南

图检索增强生成 （Graph RAG） 的发展势头强劲，并成为传统向量搜索检索方法的强大补充。这种方法利用图形数据库的结构化特性，将数据组织为节点和关系，以增强检索信息的深度和上下文性。

图形擅长以结构化方式表示和存储异构和互连的信息，轻松捕获不同数据类型的复杂关系和属性。相比之下，向量数据库经常难以处理此类结构化信息，因为它们的优势在于通过高维向量处理非结构化数据。在您的 RAG 应用程序中，您可以通过非结构化文本将结构化图形数据与向量搜索相结合，以实现两全其美的效果，这正是我们将在这篇博文中所做的。

知识图谱很棒，但如何创建知识图谱呢？构建知识图谱通常是利用基于图的数据表示功能最具挑战性的步骤。它涉及收集和构建数据，这需要对域和图形建模有深入的了解。为了简化这一过程，我们一直在试验 LLM。LLM 对语言和上下文有深刻的理解，可以自动化知识图谱创建过程的重要部分。通过分析文本数据，这些模型可以识别实体，了解它们之间的关系，并建议如何在图形结构中最好地表示它们。作为这些实验的结果，我们在 LangChain 中添加了第一个版本的图构建模块，我们将在这篇博文中演示。

## Neo4j 环境设置

您需要按照此博客文章中的示例设置 Neo4j 实例。最简单的方法是在 [Neo4j Aura]（https://neo4j.com/cloud/platform/aura-graph-database/） 上启动一个免费实例，它提供 Neo4j 数据库的云实例。或者，您也可以通过下载 Neo4j 桌面应用程序并创建本地数据库实例来设置 Neo4j 数据库的本地实例。

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-"
os.environ["NEO4J_URI"] = "bolt://18.212.197.205:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "tendencies-meanings-enlistment"

graph = Neo4jGraph()

此外，您必须提供 [OpenAI 密钥]（https://openai.com/），因为我们将在这篇博文中使用他们的模型。
## 数据摄取
对于此演示，我们将使用 Elizabeth I 的 Wikipedia 页面。我们可以利用 [LangChain loaders]() 从 Wikipedia 无缝获取和拆分文档。

In [ ]:
# Read the wikipedia article
raw_documents = WikipediaLoader(query="Elizabeth I").load()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Now it's time to construct a graph based on the retrieved documents. For this purpose, we have implemented an `LLMGraphTransformermodule` that significantly simplifies constructing and storing a knowledge graph in a graph database.

In [ ]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125") # gpt-4-0125-preview occasionally has issues
llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

您可以定义希望知识图谱生成链使用的 LLM。目前，我们仅支持来自 OpenAI 和 Mistral 的函数调用模型。但是，我们计划在未来扩大 LLM 的选择范围。在此示例中，我们使用的是最新的 GPT-4。请注意，生成的图形的质量在很大程度上取决于您使用的模型。从理论上讲，您总是希望使用功能最强大的。LLM 图形转换器返回图形文档，这些文档可以通过 'add_graph_documents' 方法导入到 Neo4j。'baseEntityLabel' 参数为每个节点分配一个额外的 '__Entity__' 标签，从而增强索引和查询性能。'include_source' 参数将节点链接到其原始文档，从而促进数据可追溯性和上下文理解。

您可以使用 yfiles 可视化检查生成的图形。

In [ ]:
# 直接显示给定 Cypher 查询生成的图形
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

## RAG 的混合检索
生成图形后，我们将使用一种混合检索方法，将向量和关键字索引与 RAG 应用程序的图形检索相结合。

![检索]（https://raw.githubusercontent.com/tomasonjo/blogs/master/graphhybrid.png）

该图说明了一个检索过程，该过程从用户提出问题开始，然后被定向到 RAG 检索器。该检索器使用关键字和向量搜索来搜索非结构化文本数据，并将其与从知识图谱中收集的信息相结合。由于 Neo4j 同时具有关键字和向量索引，因此您可以使用单个数据库系统实现所有三个检索选项。从这些来源收集的数据被馈送到 LLM 中，以生成并提供最终答案。
## 非结构化数据检索器
您可以使用 Neo4jVector.from_existing_graph 方法将关键字和向量检索添加到文档中。此方法为混合搜索方法配置关键字和向量搜索索引，以标记为 Document 的节点为目标。此外，如果缺少文本嵌入值，它会计算文本嵌入值。

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

然后可以使用 similarity_search 方法调用 vector 索引。
## 图形检索器
另一方面，配置图形检索更复杂，但提供了更多的自由度。在此示例中，我们将使用全文索引来识别相关节点，然后返回它们的直接邻域。

![图表]（https://raw.githubusercontent.com/tomasonjo/blogs/master/neighbor.png）

图形检索器首先识别输入中的相关实体。为简单起见，我们指示 LLM 识别人员、组织和位置。为此，我们将使用 LCEL 和新添加的 'with_structured_output' 方法来实现这一目标。

In [ ]:
# Retriever

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


让我们测试一下：

In [1]:
entity_chain.invoke({"question": "阿米莉亚·埃尔哈特 （Amelia Earhart） 在哪里出生？"}).names

NameError: name 'entity_chain' is not defined

太好了，现在我们可以检测问题中的实体，让我们使用全文索引将它们映射到知识图谱。首先，我们需要定义一个全文索引和一个函数，该函数将生成允许一些拼写错误的全文查询，我们在这里不会详细介绍。

In [ ]:
def generate_full_text_query(input: str) -> str:
    """为给定的输入字符串生成全文搜索查询。

    此函数构造适合全文搜索的查询字符串。
    它通过将输入字符串拆分为单词并附加一个
    相似度阈值（~2 个更改的字符）与每个单词匹配，然后组合
    他们使用 AND 运算符。用于映射用户问题中的实体
    添加到数据库值中，并允许一些拼写错误。"""
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

The `structured_retriever` function starts by detecting entities in the user question. Next, it iterates over the detected entities and uses a Cypher template to retrieve the neighborhood of relevant nodes. Let's test it out!

In [ ]:
print(structured_retriever("Who is Elizabeth I?"))

Elizabeth I - RULED -> England
Elizabeth I - RULED -> Ireland
Elizabeth I - BELONGED_TO -> House Of Tudor
Elizabeth I - PARENT -> Henry Viii
Elizabeth I - PARENT -> Anne Boleyn
Henry Viii - SPOUSE -> Anne Boleyn
Henry Viii - PARENT -> Edward Vi
Edward Vi - BEQUEATHED_CROWN_TO -> Lady Jane Grey
Edward Vi - IGNORED_CLAIMS_OF -> Mary
Edward Vi - IGNORED_CLAIMS_OF -> Elizabeth I
Mary - IMPRISONED -> Elizabeth I
Elizabeth I - DEPENDED_ON -> William Cecil
Elizabeth I - CREATED_TITLE -> Baron Burghley
Elizabeth I - SUCCEEDED_BY -> James Vi Of Scotland
Mary, Queen Of Scots - PARENT -> James Vi Of Scotland
Elizabeth I - DEPENDED_ON -> Francis Walsingham
Elizabeth - WAR -> Spain
Elizabeth - WAR -> Netherlands
Elizabeth - WAR -> France
Elizabeth - WAR -> Ireland
Elizabeth - LEAD -> William Shakespeare
Elizabeth - LEAD -> Christopher Marlowe
Elizabeth - LEAD -> Francis Drake
Elizabeth - LEAD -> Walter Raleigh
Elizabeth - DEFEAT -> Spanish Armada
Elizabeth I - SUCCESSOR -> James Vi Of Scotland
Henr

## 最终检索器
正如我们在开头提到的，我们将组合 unstructured 和 graph retriever 来创建将传递给 LLM 的最终上下文。

In [ ]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

当我们处理 Python 时，我们可以简单地使用 f 字符串连接输出。
## 定义 RAG 链
我们已经成功实施了 RAG 的检索组件。首先，我们将介绍允许对话式后续问题的查询重写部分。

In [ ]:
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

接下来，我们介绍一个提示，它利用集成的混合检索器提供的上下文来生成响应，从而完成 RAG 链的实现。

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

Finally, we can go ahead and test our hybrid RAG implementation.

In [ ]:
chain.invoke({"question": "Which house did Elizabeth I belong to?"})

Search query: Which house did Elizabeth I belong to?


'Elizabeth I belonged to the House of Tudor.'

Let's test a follow up question!

In [ ]:
chain.invoke(
    {
        "question": "When was she born?",
        "chat_history": [("Which house did Elizabeth I belong to?", "House Of Tudor")],
    }
)

Search query: When was Elizabeth I born?


'Elizabeth I was born on 7 September 1533.'